In [0]:
-- SILVER: cleaned user data with strict null removal (streaming)
CREATE OR REFRESH STREAMING TABLE users_silver
COMMENT "Cleaned user data with strict null removal"
AS
SELECT DISTINCT
    id,
    TRIM(name)                          AS name,
    LOWER(TRIM(username))               AS username,
    LOWER(TRIM(email))                  AS email,
    TRIM(phone)                         AS phone,
    LOWER(TRIM(gender))                 AS gender,
    LOWER(TRIM(food_preference))        AS food_preference,
    TRIM(locality)                      AS locality,
    TRIM(address)                       AS address,
    LOWER(TRIM(role))                   AS role,
    is_active,
    created_at,
    updated_at
FROM (
    SELECT
        id,
        NULLIF(TRIM(name),            '') AS name,
        NULLIF(TRIM(username),        '') AS username,
        NULLIF(TRIM(email),           '') AS email,
        NULLIF(TRIM(phone),           '') AS phone,
        NULLIF(TRIM(gender),          '') AS gender,
        NULLIF(TRIM(food_preference), '') AS food_preference,
        NULLIF(TRIM(locality),        '') AS locality,
        NULLIF(TRIM(address),         '') AS address,
        NULLIF(TRIM(role),            '') AS role,
        is_active,
        created_at,
        updated_at
    FROM STREAM(users_bronze)
)
WHERE id            IS NOT NULL
  AND name          IS NOT NULL
  AND username      IS NOT NULL
  AND email         IS NOT NULL
  AND phone         IS NOT NULL
  AND gender        IS NOT NULL
  AND food_preference IS NOT NULL
  AND locality      IS NOT NULL
  AND address       IS NOT NULL
  AND role          IS NOT NULL
  AND is_active     IS NOT NULL
  AND created_at    IS NOT NULL
  AND updated_at    IS NOT NULL;

In [0]:
CREATE OR REFRESH LIVE TABLE orders_silver
COMMENT "Cleaned and deduplicated order data"
AS
WITH cleaned AS (
    SELECT
        NULLIF(TRIM(order_id), '') AS order_id,
        NULLIF(TRIM(restaurant_name), '') AS restaurant_name,
        NULLIF(total_amount, 0) AS total_amount,
        NULLIF(items_count, 0) AS items_count,
        CAST(timestamp AS TIMESTAMP) AS timestamp,
        NULLIF(TRIM(user_id), '') AS user_id,
        NULLIF(TRIM(has_duplicate), '') AS has_duplicate
    FROM LIVE.raw_data_bronze
)
SELECT
    order_id,
    restaurant_name,
    total_amount,
    items_count,
    timestamp,
    user_id,
    has_duplicate
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY order_id ORDER BY timestamp DESC) AS rn
    FROM cleaned
)
WHERE rn = 1
  AND order_id IS NOT NULL
  AND user_id IS NOT NULL
  AND restaurant_name IS NOT NULL
  AND total_amount IS NOT NULL
  AND items_count IS NOT NULL
  AND timestamp IS NOT NULL;